In [129]:
!pip install geocoder
!pip install folium==0.5.0
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [130]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print("Successfully imported.")

Successfully imported.


In [131]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup = BeautifulSoup(data, 'html.parser')

In [132]:
# create a list to store neighborhood data
neighborhoodList = []
for row in soup("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
del neighborhoodList[0]
berlin_df = pd.DataFrame({"Neighborhood": neighborhoodList})
berlin_df.head()

,Neighborhood
0,Adlershof
1,Afrikanisches Viertel
2,Alt-Hohenschönhausen
3,Alt-Treptow
4,Altglienicke


In [133]:
#the number of rows of the dataframe
berlin_df.shape

(97, 1)

In [134]:
# define a function to get coordinates
def get_latlng(neighborhood):
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [135]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

In [136]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [137]:
# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']

In [138]:
# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head()

(97, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.437790,13.54778
1,Afrikanisches Viertel,52.558269,13.33389
2,Alt-Hohenschönhausen,52.547060,13.50055
3,Alt-Treptow,52.493500,13.45711
4,Altglienicke,52.420060,13.53969


In [139]:
# save the DataFrame as CSV file
berlin_df.to_csv("berlin_df.csv", index=False)

In [140]:
# get the coordinates of Berlin
address = 'Berlin, Germany'
geolocator = Nominatim(user_agent="bainan.liu@gmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin, Germany {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin, Germany 52.5170365, 13.3888599.


In [141]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

In [142]:
# save the map as HTML file
map_berlin.save('map_berlin.html')

In [143]:
# define Foursquare Credentials and Version
CLIENT_ID = 'KP23O2U4XKDE12JWQ52IANQIJUO01POWJIVTAHI0WGC5IX50' # your Foursquare ID
CLIENT_SECRET = 'FWF31DOXLUP3TWZVQRAVFIRPXG5SCFXK2U3HSHUIEW1HQAGH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KP23O2U4XKDE12JWQ52IANQIJUO01POWJIVTAHI0WGC5IX50
CLIENT_SECRET:FWF31DOXLUP3TWZVQRAVFIRPXG5SCFXK2U3HSHUIEW1HQAGH


In [144]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    
    #create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [145]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5420, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlershof,52.43779,13.54778,Mia Toscana,52.438327,13.549573,Italian Restaurant
1,Adlershof,52.43779,13.54778,Rossmann,52.438320,13.548168,Drugstore
2,Adlershof,52.43779,13.54778,dm-drogerie markt,52.437625,13.547692,Drugstore
3,Adlershof,52.43779,13.54778,REWE,52.437855,13.545603,Supermarket
4,Adlershof,52.43779,13.54778,EDEKA,52.438345,13.548332,Supermarket


In [146]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adlershof,43,43,43,43,43,43
Afrikanisches Viertel,89,89,89,89,89,89
Alt-Hohenschönhausen,62,62,62,62,62,62
Alt-Treptow,100,100,100,100,100,100
Altglienicke,26,26,26,26,26,26
Baumschulenweg,65,65,65,65,65,65
Biesdorf (Berlin),24,24,24,24,24,24
Blankenburg (Berlin),15,15,15,15,15,15
Blankenfelde,8,8,8,8,8,8


In [147]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 335 uniques categories.


In [148]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Italian Restaurant', 'Drugstore', 'Supermarket',
       'Sushi Restaurant', 'Trattoria/Osteria', 'Palace',
       'Korean Restaurant', 'Ice Cream Shop', 'German Restaurant',
       'Gym / Fitness Center', 'Café', 'Hotel', 'Brewery', 'Plaza',
       'Movie Theater', 'Tram Station', 'Pet Store', 'Hardware Store',
       'Big Box Store', 'Sandwich Place', 'Restaurant', 'Gas Station',
       'Soccer Field', 'Light Rail Station', 'Austrian Restaurant',
       'Park', 'Chinese Restaurant', 'Indian Restaurant',
       'African Restaurant', 'Middle Eastern Restaurant',
       'Argentinian Restaurant', 'Yemeni Restaurant',
       'Turkish Restaurant', 'Bar', 'Seafood Restaurant',
       'Scenic Lookout', 'Organic Grocery', 'Grocery Store',
       'Vietnamese Restaurant', 'Burger Joint', 'Coffee Shop',
       'French Restaurant', 'Bowling Alley', 'Hookah Bar',
       'Mediterranean Restaurant', 'Farmers Market', 'Pizza Place',
       'Electronics Store', 'Lounge', 'Fried Chicken Joint'],

In [149]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [150]:
# one hot encoding
berlin_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

print(berlin_onehot.shape)
berlin_onehot.head()

(5420, 336)


,Neighborhoods,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,

In [151]:
berlin_grouped = berlin_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(berlin_grouped.shape)
berlin_grouped.head()

(97, 336)


,Neighborhoods,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,

In [152]:
len(berlin_grouped[berlin_grouped["Shopping Mall"] > 0])

21

In [153]:
berlin_mall = berlin_grouped[["Neighborhoods","Shopping Mall"]]
berlin_mall.head()

,Neighborhoods,Shopping Mall
0,Adlershof,0.000000
1,Afrikanisches Viertel,0.000000
2,Alt-Hohenschönhausen,0.032258
3,Alt-Treptow,0.000000
4,Altglienicke,0.000000


In [154]:
# set number of clusters
kclusters = 3
berlin_clustering = berlin_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 2, 0, 0, 0, 1], dtype=int32)

In [155]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
berlin_merged = berlin_mall.copy()

# add clustering labels
berlin_merged["Cluster Labels"] = kmeans.labels_

In [156]:
berlin_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
berlin_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adlershof,0.000000,0
1,Afrikanisches Viertel,0.000000,0
2,Alt-Hohenschönhausen,0.032258,1
3,Alt-Treptow,0.000000,0
4,Altglienicke,0.000000,0


In [157]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
berlin_merged = berlin_merged.join(berlin_df.set_index("Neighborhood"), on="Neighborhood")

print(berlin_merged.shape)
berlin_merged.head() # check the last columns!

(97, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adlershof,0.000000,0,52.437790,13.54778
1,Afrikanisches Viertel,0.000000,0,52.558269,13.33389
2,Alt-Hohenschönhausen,0.032258,1,52.547060,13.50055
3,Alt-Treptow,0.000000,0,52.493500,13.45711
4,Altglienicke,0.000000,0,52.420060,13.53969


In [158]:
# sort the results by Cluster Labels
print(berlin_merged.shape)
berlin_merged.sort_values(["Cluster Labels"], inplace=True)
berlin_merged

(97, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adlershof,0.000000,0,52.437790,13.54778
66,Oberschöneweide,0.000000,0,52.462370,13.51438
65,Nikolassee,0.000000,0,52.434110,13.19973
64,Niederschönhausen,0.000000,0,52.582650,13.40362
63,Niederschöneweide,0.000000,0,52.455490,13.51328
62,Neukölln (locality),0.000000,0,52.480770,13.43541
60,Müggelheim,0.000000,0,52.413000,13.66050
59,Märkisches Viertel,0.000000,0,52.596800,13.35831
58,Moabit,0.000000,0,52.525700,13.34005
57,Mitte (locality),0.000000,0,52.521190,13.42414


In [159]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['Latitude'], berlin_merged['Longitude'], berlin_merged['Neighborhood'], berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)    
map_clusters

In [160]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [161]:
berlin_merged.loc[berlin_merged['Cluster Labels'] == 0].head()

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adlershof,0.0,0,52.43779,13.54778
66,Oberschöneweide,0.0,0,52.46237,13.51438
65,Nikolassee,0.0,0,52.43411,13.19973
64,Niederschönhausen,0.0,0,52.58265,13.40362
63,Niederschöneweide,0.0,0,52.45549,13.51328


In [162]:
berlin_merged.loc[berlin_merged['Cluster Labels'] == 1].head()

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
56,Marzahn,0.035088,1,52.54212,13.54389
2,Alt-Hohenschönhausen,0.032258,1,52.54706,13.50055
53,Malchow (Berlin),0.062500,1,52.57848,13.48300
52,Mahlsdorf,0.035714,1,52.51297,13.61185
61,Neu-Hohenschönhausen,0.048780,1,52.56512,13.50828


In [163]:
berlin_merged.loc[berlin_merged['Cluster Labels'] == 2].head()

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
45,Kreuzberg,0.010000,2,52.49382,13.38358
5,Baumschulenweg,0.015385,2,52.46669,13.48840
34,Haselhorst,0.020408,2,52.54336,13.24152
84,Tempelhof,0.010870,2,52.46076,13.38563
10,Borsigwalde,0.025974,2,52.58522,13.30788
